# Preprocesamiento

Dentro de este documento se realizará un preprocesamiento al corpus previamente definido de acuerdo con las observaciones sugeridas.

Este preprocesamiento incluye:
1. Tokenizacion
3. Preprocesamiento de stopwords
    1. Stopwords generales.
    2. Números.
    3. Palabras en inglés y otros símbolos.
2. Stemming
4. Calculo tf idf

In [1]:
#Importar elementos necesarios de las librerías
import os, shutil, re, pickle
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import PlaintextCorpusReader, stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.snowball import SpanishStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [2]:
#Funciones auxiliares

def read_files(path,stemmer=None):
    #Tokeniza un corpus usando los documentos de una ruta y devuelve una lista con el tamaño del corpus
    corpus, size = [], []
    files = os.listdir(path)
    if stemmer == None:
        #Proceso de tokenización sin stemming
        for file in files:
            texto = PlaintextCorpusReader(ruta, file)
            size.append(len(texto.words()))
            texto = texto.raw()
            corpus.append(texto)
    else:
        #Proceso de tokenización con stemming
        for file in files:
            texto = PlaintextCorpusReader(ruta, file)
            tokens = texto.words()
            size.append(len(texto.words()))
            tokens = [stemmer.stem(token) for token in tokens]
            texto = ' '.join(tokens)
            corpus.append(texto)
    return corpus, size

def tf_idf_to_dataframe(array_tf_idf,path):
    #Crea un dataframe a partir de un array que contenga el tf-idf de un corpus
    files = os.listdir(path)
    for i, doc in enumerate(array_tf_idf):
        title = files[i].replace(".txt","")
        tf_idf_single_document = list(zip(tf_idf.get_feature_names(), doc, [i for _ in range(len(doc))], [title for _ in range(len(doc))]))
        if i == 0:
            df = pd.DataFrame(tf_idf_single_document, columns=['token', 'tf_idf','id_documento','nombre_documento'])
        else:
            df2 = pd.DataFrame(tf_idf_single_document, columns=['token', 'tf_idf','id_documento','nombre_documento'])
            df = df.append(df2)
    return df

def plot_bar(title,x,y,x_name,y_name,scale=None):
    #Función auxiliar para realizar graficos de barras
    plt.bar(x, y, align='center', alpha=0.5)
    plt.xlabel(x_name)
    plt.ylabel(y_name)
    if scale != None: plt.yscale(scale)
    plt.title(title)
    plt.grid()
    plt.show()

def plot_tf_idf(array_tf_idf,path,size):
    #Grafica el tf-idf de cada documento de una ruta e imprime el tamaño de cada documento
    files = os.listdir(path)
    for i, doc in enumerate(array_tf_idf):
        document_title = files[i].replace(".txt","")
        print("Cantidad de tokens del documento de {}: {}".format(document_title,size[i]))
        x_axis = [i for i in range(len(doc))]
        plot_bar(document_title,x_axis,doc,'Llaves del token','Tf-idf','log')
        
def plot_general_tf_idf(array_tf_idf,path,tokens):
    #Grafica el tf-idf máximo y promedio de cada token en el corpus
    files = os.listdir(path)
    max_tf_idf, mean_tf_idf = [0 for _ in range(tokens)], [0 for _ in range(tokens)]
    for i, doc in enumerate(array_tf_idf):
        tf_idf_single_document = list(zip(tf_idf.get_feature_names(), doc, [i for _ in range(len(doc))]))
        document_title = files[i].replace(".txt","")
        max_tf_idf = [max(max_tf_idf[j],doc[j]) for j in range(tokens)]
        mean_tf_idf = [mean_tf_idf[j] + doc[j] for j in range(tokens)]
    mean_tf_idf = [mean_tf_idf[j]/(i+1) for j in range(tokens)]
    x_axis = [i for i in range(tokens)]
    plot_bar('Tf-idf máximo',x_axis,max_tf_idf,'Llave del token','Tf-idf','log')
    plot_bar('Tf-idf promedio',x_axis,mean_tf_idf,'Llave del token','Tf-idf','log')

def show_top_tokens(df,path,n):
    #Muestra los tokens con mayor tf-idf de cada documento
    files = os.listdir(path)
    for text in files:
        title = text.replace(".txt","")
        print(df[df.nombre_documento == title].head(n),'\n\n')

def get_regex_corpus_list(path,regex):
    #obtiene una lista de los tokens de un corpus que coincidan con una expresión regular
    corpus, regex_list = [], []
    files = os.listdir(path)
    for file in files:
        texto = PlaintextCorpusReader(ruta, file)
        tokens = texto.words()
        regex_list += [token for token in tokens if regex.match(token)]
    regex_list = list(set(regex_list))
    return regex_list

# Preprocesamiento
## Fase 1. y Fase 2. Tokenización y Stemming

In [3]:
#Ruta del corpus
ruta = "D:/Documents/Documentos Universidad/Noveno/Proyecto de grado/textos"
stemmer = SpanishStemmer()

#Tokenización y stemming
corpus, tamaño = read_files(ruta,stemmer)

## Fase 3. Preprocesamiento stopwords
### Fase 3.A. Stopwords generales
Con el fin de determinar posibles stopwords, se sugiere iniciar el analisis eliminando aquellos tokens sin mucha relevancia y que son frecuentes en el lenguaje español (pronombres, artículos, preposiciones, etc). Para ello se utilizará una lista como referencia una lista de stopwords que provee NLTK.

In [4]:
#Stopwords de referencia
stopwords.words('spanish')

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [5]:
#Lista de stopwords (palabras vacías)
stopwords_generales = ['de','la','que','el','en','y','a','los','del','se','las','por','un',
                       'para','con','no','una','su','al','lo','como','mas','pero','sus','le',
                       'ya','o','este','si','porque','esta','entre','cuando','muy','sin','tus',
                       'sobre','también','me','hasta','hay','donde','quien','desde','todo','estas',
                       'nos','durante','uno','les','ni','ese','eso','contra','otros','mi','yo',
                       'e','ante','usted','ellos','ella','nosotros','mis','te','ti','tu','vosotros',
                       'mío','mía','míos','mías','tuyo','suyo','nuestro','vuestro','soy','eres',
                       'es','os','nada','muchos','cual','poco','esto','antes','alguno','otro','ud',
                       'otra','tanto','he','has','ha','hemos','habéis','hube','han','esa','esos',
                       'esas','algo','somos','son','sois','sea','seas','seamos','seáis','sean',
                       'ser','era','eras','éramos','eran','erais','fui','fuiste','fue','fuimos',
                       'fueron','fuera','fuese']

#Aplicar stemming a las stopwords
stopwords_generales = [stemmer.stem(token) for token in stopwords_generales]

### Fase 3.B. Números
En esta fase se filtraran cualquier token que empiece con un número

In [6]:
#Expresión regular para filtrar los tokens que empiecen con algún número
regex = re.compile('^[0-9]')

#Lista con los tokens que empiezan con algún npumero
stopwords_numeros = get_regex_corpus_list(ruta,regex)

#Aplicar stemming a las stopwords
stopwords_numeros = [stemmer.stem(token) for token in stopwords_numeros]

#Juntar las listas de stopwords
stopwords_numeros += stopwords_generales

### Fase 3.C. Palabras en inglés y otros
Aquí se procederá a eliminar las palabras en otro idioma diferente al español cuya frecuencia es bastante alta, y componentes de dominios o hipervinculos.

In [7]:
#Lista de otras stopwords
final_stopwords  = ['aka','s','inc','ltda','@','com','esrb','co','ncs','http','https',
                     'www', 'net', 'drm','book','books','reader', 'editions', 'tagus',
                    'ebook','ebooks','link','pub','eee','aaa', 'group', 'limited', 'hipervinculos',
                    'cgc', 'help','efta','periscope','cci','url']

#Aplicar stemming a las stopwords
final_stopwords = [stemmer.stem(token) for token in final_stopwords]

#Juntar las listas de stopwords
final_stopwords += stopwords_numeros

## Fase 4. Cálculo tf-idf
En esta fase se calculará el tf-idf de las utilizando las stopwords previamente definidas.

In [8]:
tf_idf = TfidfVectorizer(stop_words = final_stopwords)

#Matriz tf-idf
result = tf_idf.fit_transform(corpus)

In [9]:
#Reducción de dimensionalidad
array_tf_idf = result.toarray()

#Crear un dataframe con el arreglo de tf-idf
df_final = tf_idf_to_dataframe(array_tf_idf,ruta)

#Ordena el dataframe descendentemente de acuerdo con el tf-idf
df_final = df_final.sort_values(by='tf_idf', ascending=False)

In [10]:
#Muestra los 20 tokens con mayor tf-idf de cada documento del corpus
show_top_tokens(df_final,ruta,20)

          token    tf_idf  id_documento   nombre_documento
111        adob  0.420085             0  adobe-condiciones
4116    softwar  0.384952             0  adobe-condiciones
4024    servici  0.319339             0  adobe-condiciones
772       cloud  0.209988             0  adobe-condiciones
904   condicion  0.209668             0  adobe-condiciones
1100    creativ  0.193373             0  adobe-condiciones
989      conten  0.192527             0  adobe-condiciones
3571       pued  0.139841             0  adobe-condiciones
1126    cualqui  0.136191             0  adobe-condiciones
1256     derech  0.116736             0  adobe-condiciones
836      compañ  0.113453             0  adobe-condiciones
4540        uso  0.112579             0  adobe-condiciones
1574      emple  0.109840             0  adobe-condiciones
4616    version  0.105600             0  adobe-condiciones
1136      cuent  0.099005             0  adobe-condiciones
500         bet  0.098587             0  adobe-condicion

In [11]:
#Top 60 tokens con mayor tf-idf en todo el corpus
df_final.head(60)

,token,tf_idf,id_documento,nombre_documento
1209,deez,0.905419,7,deezer
3295,pinterest,0.900739,33,pinterest-condiciones
4150,spotify,0.888717,41,spotify-condiciones
4353,textbrok,0.833278,44,textbroker-autores
4353,textbrok,0.812786,45,textbroker-clientes
111,adob,0.812348,1,adobe-privacidad
2955,netflix,0.811244,26,netflix-condiciones
723,chrom,0.796939,4,chrome-privacidad
4150,spotify,0.789254,42,spotify-privacidad
4478,uber,0.785742,50,uber-privacidad


In [12]:
#Medidas de centralidad y cuartiles
df_final.describe()

,tf_idf,id_documento
count,265440.000000,265440.00000
mean,0.002318,27.50000
std,0.014339,16.16326
min,0.000000,0.00000
25%,0.000000,13.75000
50%,0.000000,27.50000
75%,0.000000,41.25000
max,0.905419,55.00000


In [13]:
#Guardar el tf-idf
filename = 'tf-idf_model.pkl'
pickle.dump(tf_idf, open(filename, 'wb'))